
This will be the classes used for migration. The first functionality of the classes will be to generate an inventory assets to be migrated.

The entry class for migration is Experiment. The design intent is that the `MigrateRegisteredModel` class does a `meta_sync` with mlflow. All dependent Experiments will then have `MigrateExperiment` classes that do `meta_sync`.

This migration class will cache the state of the registered model and dependent experiments. Changes made after instantiation will not be captured.

**NOTE**: This migration code was developed in **DBR 16.1 ML** so that we are using the most current convention on ABC in python 3.12.

**FEATURES**:
 * This migration code does allow for the configuration of migrated experiments having UC Volumes archive locations
 * This migration code will attempt to migrate orphaned Registered Models (That is, registered models that have had their source training experiment run deleted)
 * This migration code is written to allow for different clients between registered model and experiment. However, it currently does not support having experiments from multiple workspaces. 

**CONSTRAINTS**: To facilitate automation, there are constraints imposed to simplify usability.
 * Source Experiments are **NOT** upgraded. Instead, there is a migrated Target Experiment that has the suffix, `_uc`.
 * Source Experiments must be a workspace (instead of notebook) experiment. 
 * Notebook Training Sources are **NOT** migrated. Instead, the notebook reference is passed into the migrated experiment run. This ensures tracibility back to the source notebook. However, that source notebook will still need to have it's training run code updated which is not in scope of this migration tool.
 * Since MLFlow works on global configuraitions, it is problematic to workin both UC and WS registry URIs at the same time. To manage this, we'll explicitly write a context manager for each session call. However, running multiple migrations in the same session will become problematic due to overwriting the global configuration during migration runs. Thus, each migration will need to have it's own python environment when migrating or the migration should be run in series.



In [0]:
#%pip install --upgrade "mlflow-skinny[databricks]==2.20.0"
%pip install --upgrade "mlflow[databricks]==2.20.0"
%pip install --upgrade "scikit-learn==1.6.1"
dbutils.library.restartPython()

In [0]:
from abc import ABC, abstractmethod
from typing import override
from functools import cached_property
from mlflow.tracking import MlflowClient
from contextlib import contextmanager
import mlflow
from mlflow.entities.model_registry.registered_model import RegisteredModel
from mlflow.entities.experiment import Experiment

class MLFlowEntityBase(ABC):
    """Base Class when working with MLFlow Entities that may have custom tracking_uri and/or registry_uri"""

    _tracking_uri: str = "databricks"
    _registry_uri: str = "databricks-uc"

    @contextmanager
    def mlflow_client(self):
        global_tracking_uri = mlflow.get_tracking_uri()
        global_registry_uri = mlflow.get_registry_uri()
        mlflow.set_tracking_uri(self._tracking_uri)
        mlflow.set_registry_uri(self._registry_uri)
        try:
            yield MlflowClient(tracking_uri=self._tracking_uri,
                               registry_uri=self._registry_uri)
        finally:
            mlflow.set_tracking_uri(global_tracking_uri)
            mlflow.set_registry_uri(global_registry_uri)

In [0]:
class ExperimentRun(MLFlowEntityBase):

    _run_id: str

    @property
    def run_id(self):
        return self._run_id

class SrcExperimentRun(ExperimentRun):

    def __init__(self, run_id: str):
        self._run_id = run_id

class TgtExperimentRun(ExperimentRun):

    def __init__(self, run_id: str):
        self._run_id = run_id
    
    def initialize_tgt_run(self):
        pass

In [0]:
from abc import ABC, abstractmethod
from typing import override
from functools import cached_property
from mlflow.tracking import MlflowClient
from contextlib import contextmanager
import mlflow
from mlflow.entities.model_registry.registered_model import RegisteredModel
from mlflow.entities.experiment import Experiment
from mlflow.entities.run import Run

# NOTE: The migrate convention is to have the experiment_name for the src experiments be the same experiment name + "_migrate"
#       There is no intention to make this configurable by experiement, you must manually rename after migration if alternate name is required.
# There is intention to map experiment runs with the same details they had in the source experiment

class ModelExperiment(MLFlowEntityBase):

    _experiment_id: str
    _experiment_name: str
    _artifact_location: str

    @property
    def experiment_id(self):
        return self._experiment_id

    @property
    def experiment_name(self):
        return self._experiment_name

    @property
    def artifact_location(self):
        return self._artifact_location

    def _set_experiment_id_name(self, experiment_id: str=None,
                                      experiment_name: str=None,
                                      artifact_location: str=None):
        with self.mlflow_client() as client:
            if experiment_id: 
                experiment = client.get_experiment(experiment_id)
                print("Successfully identified experiment: " + experiment.name)
                self._experiment_name = experiment.name
                self._experiment_id = experiment.experiment_id
            elif experiment_name:
                experiment_name = experiment_name[10:] if experiment_name[:10] == "/Workspace" else experiment_name
                try:
                    if artifact_location:
                        self._experiment_id = client.create_experiment(name=experiment_name,
                                                                       artifact_location=artifact_location)
                    else:
                        self._experiment_id = client.create_experiment(name=experiment_name)
                    print("Successfully created experiment: " + experiment_name)
                    self._experiment_name = experiment_name
                except:
                    experiment = client.get_experiment_by_name(experiment_name)
                    print("Identified existing experiment: " + experiment.name)
                    self._experiment_id = experiment.experiment_id
                    self._experiment_name = experiment_name
            else:
                raise BaseException("Must supply either an experiment_id or an experiment_name")

    @property
    def experiment(self) -> Experiment:
        with self.mlflow_client() as client:
            return client.get_experiment(self.experiment_id)

    def _get_experiment_runs(self) -> [Run]:
        """To find all experiment runs within this experiment"""
        with self.mlflow_client() as client:
            return client.search_runs(experiment_ids=[self.experiment_id,])
        
    def _set_artifact_location(self, artifact_location):
        if artifact_location:
            artifact_location = artifact_location if artifact_location[:5]=="dbfs:" else "dbfs:" + artifact_location
            name = self._experiment_name.split('/')[-1]
            if artifact_location[-1] == "/":
                self._artifact_location = f"{artifact_location}{name}"
            else:
                self._artifact_location = f"{artifact_location}/{name}"

    @abstractmethod
    def experiment_runs(self) -> [ExperimentRun]:
        """This method must be overridden in subclasses."""
        pass

    def status(self):
        self._exp_status()
        self._exp_run_status()

    @abstractmethod
    def _exp_status(self):
        pass

    @abstractmethod
    def _exp_run_status(self):
        pass

class SrcModelExperiment(ModelExperiment):

    def __init__(self, experiment_id: str,
                       tracking_uri: str = None):
        self._tracking_uri = tracking_uri
        self._set_experiment_id_name(experiment_id=experiment_id)

    @override
    @cached_property
    def experiment_runs(self) -> [ExperimentRun]:
        return [SrcExperimentRun(run_id = r.info.run_id) for r in self._get_experiment_runs()]

    @override
    def _exp_status(self):
        print(f'    Source Experiment, {self.__class__.__name__}, name: {self.experiment_name}')

    @override
    def _exp_run_status(self):
        for i, r in enumerate(self.experiment_runs):
            print(f'      Source Experiment Run {str(i).zfill(4)}, {r.__class__.__name__}, id: {r._run_id}')

class TgtModelExperiment(ModelExperiment):

    _src_exp: SrcModelExperiment

    def __init__(self, src_exp: SrcModelExperiment,
                       tracking_uri: str = None,
                       artifact_location: str = None):
        self._src_exp = src_exp
        self._experiment_name = src_exp.experiment_name + "_uc"
        self._tracking_uri = tracking_uri
        self._set_artifact_location(artifact_location)
        self._set_experiment_id_name(experiment_name=self._experiment_name,
                                     artifact_location=self._artifact_location)
    
    @override
    def experiment_runs(self) -> [ExperimentRun]:
        """This method must be overridden in subclasses."""
        return []

    @override
    def _exp_status(self):
        print(f'Target Experiment, {self.__class__.__name__}, name: {self.experiment_name}')

    @override
    def _exp_run_status(self):
        pass

    def initialize_tgt_entities(self):
        self.initialize_tgt_entities()
        for r in self.experiment_runs():
            r.initialize_tgt_run()

    def initialize_tgt_exp(self):
        pass


In [0]:
from mlflow.entities.model_registry import ModelVersion

class RegisteredModelVersion(MLFlowEntityBase):

    name: str
    version: str

    def __init__(self, registered_model_name: str,
                       registered_model_version: str,
                       registry_uri: str):
        self._registry_uri = registry_uri
        self._set_registered_model_name_versions(registered_model_name,
                                                 registered_model_version)

    def _set_registered_model_name_versions(self, registered_model_name: str,
                                                  registered_model_version: str):
        self.name = registered_model_name
        self.version = registered_model_version
        _ = self.ModelVersion

    @cached_property
    def ModelVersion(self) -> ModelVersion:
        with self.mlflow_client() as client:
            return client.get_model_version(name=self.name,
                                            version=self.version)
    
    @abstractmethod
    def status(self):
        pass


class WsRegisteredModelVersion(RegisteredModelVersion):
    
    @override
    def status(self):
        print(f'      Source Registered Model Verion, {self.__class__.__name__}, name: {self.name}-{self.version}')


class UcRegisteredModelVersion(RegisteredModelVersion):

    @override
    def status(self):
        print(f'Target Registered Model, {self.__class__.__name__}, name: {self.registered_model_name}')

In [0]:
class RegisteredModelBase(MLFlowEntityBase):

    _rm_name: str
    _artifact_location: str

    @cached_property
    def registered_model_name(self):
        """Will get or create a registered model to ensure that the registered model exists."""
        with self.mlflow_client() as client:
            try:
                client.create_registered_model(name=self._rm_name)
                print("Successfully created registered model: " + self._rm_name)
            except:
                print("Identified existing registered model: " + client.get_registered_model(name=self._rm_name).name)
        return self._rm_name

    @property
    def registered_model(self) -> RegisteredModel:
        with self.mlflow_client() as client:
            return client.get_registered_model(name=self.registered_model_name)
    
    def _get_registered_model_versions(self) -> [ModelVersion]:
        with self.mlflow_client() as client:
            return client.search_model_versions(filter_string=f"name='{self.registered_model_name}'")

    @abstractmethod
    def model_versions(self) -> [WsRegisteredModelVersion]:
        """This method must be overridden in subclasses."""
        pass

    def _get_experiments(self) -> [Experiment]:
        """To find all parent experiments for a registered model."""
        with self.mlflow_client() as client:
            model_versions = client.search_model_versions(filter_string=f"name='{self.registered_model_name}'")
            run_ids = set([mv.run_id for mv in model_versions])
            experiment_ids = set([client.get_run(r).info.experiment_id for r in run_ids])
            return [client.get_experiment(e) for e in experiment_ids]

    @abstractmethod
    def model_experiments(self) -> [ModelExperiment]:
        """This method must be overridden in subclasses."""
        pass

    def status(self):
        self._rm_status()
        self._rm_ver_status()
        self._exp_status()
    
    @abstractmethod
    def _rm_status(self):
        pass
    
    @abstractmethod
    def _rm_ver_status(self):
        pass

    @abstractmethod
    def _exp_status(self):
        pass

In [0]:
class WsRegisteredModel(RegisteredModelBase):

    def __init__(self, registered_model_name: str,
                       tracking_uri = "databricks",
                       registry_uri = "databricks"):
        self._rm_name = registered_model_name
        self._tracking_uri = tracking_uri
        self._registry_uri = registry_uri

    @override
    @cached_property
    def model_experiments(self) -> [SrcModelExperiment]:
        return [SrcModelExperiment(experiment_id=e.experiment_id,
                                   tracking_uri=self._tracking_uri) for e in self._get_experiments()]

    @override
    @cached_property
    def model_versions(self) -> [WsRegisteredModelVersion]:
        return [WsRegisteredModelVersion(registered_model_name = v.name,
                                         registered_model_version = v.version,
                                         registry_uri = self._registry_uri) for v in self._get_registered_model_versions()]

    @override
    def _rm_status(self):
        print(f'Source Registered Model, {self.__class__.__name__}, name: {self.registered_model_name}')
    
    @override
    def _rm_ver_status(self):
        for v in self.model_versions:
            v.status()
        pass

    @override
    def _exp_status(self):
        for e in self.model_experiments:
            e.status()
    

class UcRegisteredModel(RegisteredModelBase):

    _src_rm: WsRegisteredModel
    _artifact_location: str

    def __init__(self, registered_model_name: str,
                       src_rm: WsRegisteredModel,
                       tracking_uri = "databricks",
                       registry_uri = "databricks-uc",
                       artifact_location: str = None):
        self._rm_name = registered_model_name
        self._src_rm = src_rm
        self._tracking_uri = tracking_uri
        self._registry_uri = registry_uri
        self._artifact_location = artifact_location

    @override
    @cached_property
    def model_experiments(self) -> [TgtModelExperiment]:
        """Will get or create name generates TgtModelExperiment for all parent experiments"""
        # Will require iterating through src_rm.
        # This iteration will take the following form:
        #        Identify all src_rm.model_experiments and create experiment_name
        #           Experiement name will be deterministically assigned reusing the name of the existing experiment
        return [TgtModelExperiment(src_exp=e,
                                   tracking_uri=self._tracking_uri,
                                   artifact_location=self._artifact_location)
                for e in self._src_rm.model_experiments]

    @override
    @property
    def model_versions(self) -> [WsRegisteredModelVersion]:
        return [UcRegisteredModelVersion(registered_model_name = v.name,
                                         registered_model_version = v.version,
                                         registry_uri = self._registry_uri) for v in self._get_registered_model_versions()]

    @override
    def _rm_status(self):
        print(f'Target Registered Model, {self.__class__.__name__}, name: {self.registered_model_name}')
    
    @override
    def _rm_ver_status(self):
        for v in self.model_versions:
            v.status()
        pass

    @override
    def _exp_status(self):
        print("TODO")
    
    def initialize_tgt_entities(self):
        self.initialize_tgt_rm()
        for v in self.model_experiments:
            v.initialize_tgt_exp()

    def initialize_tgt_rm(self):
        pass

In [0]:
class RegisteredModelMigration:

    src_rm: WsRegisteredModel
    tgt_rm: UcRegisteredModel

    def __init__(self, ws_registered_model_name: str,
                       uc_registered_model_name: str,
                       ws_tracking_uri: str = "databricks",
                       ws_registry_uri: str = "databricks",
                       uc_tracking_uri: str = "databricks",
                       uc_registry_uri: str = "databricks-uc",
                       uc_artifact_location: str = None):
        self.src_rm = WsRegisteredModel(registered_model_name=ws_registered_model_name,
                                        tracking_uri=ws_tracking_uri,
                                        registry_uri=ws_registry_uri)
        self.tgt_rm = UcRegisteredModel(registered_model_name=uc_registered_model_name,
                                        src_rm = self.src_rm,
                                        tracking_uri=uc_tracking_uri,
                                        registry_uri=uc_registry_uri,
                                        artifact_location=uc_artifact_location)

    @property
    def src_exps(self) -> [SrcModelExperiment]:
        return self.src_rm.model_experiments

    @property
    def tgt_exps(self) -> [TgtModelExperiment]:
        return self.tgt_rm.model_experiments

    def status(self):
        self.src_rm.status()
        self.tgt_rm.status()

    def initialize_tgt_entities(self):
        # This will initialize the following target entities:
        #   - UC Registered Models
        #   - Target Experiments
        #   - Target Experiment Runs
        # NOTE: This initialization does not attempt any content migration. This is written as a separate step to ensure UC/WS Permissions.
        self.tgt_rm.initialize_tgt_entities()


### Migration Instantiation

During Migration instantaion the following will be executed:
 - All source entities will be identified
 - Target classes will be instantiated, but no corresponding entity will be retrieved or instantiated.

During instantiation we should expect stdout messages that show all the source identified entities.

The following methods will be called:
 * 

In [0]:
ws_registered_model_name = 'orphaned'
uc_registered_model_name = 'main.default.orphaned'
uc_artifact_location = "dbfs:/Volumes/main/default/experiments"


migration = RegisteredModelMigration(ws_registered_model_name=ws_registered_model_name,
                                     uc_registered_model_name=uc_registered_model_name,
                                     uc_artifact_location=uc_artifact_location)

In [0]:
migration.status()

In [0]:
migration.initialize_tgt_entities()

In [0]:
tgt_exp = migration.tgt_rm.model_experiments[0]
self = tgt_exp
self._artifact_location

In [0]:
%python
from mlflow.tracking import MlflowClient

# Example usage
client = MlflowClient()

mlflow.set_registry_uri('databricks-uc')

In [0]:
try:

?client.create_registered_model

In [0]:
%python
from mlflow.tracking import MlflowClient

# Example usage
client = MlflowClient()

In [0]:
?client.create_experiment